In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA


In [2]:
df = pd.read_csv('gurgoan_properties_post_feature_selection V21.csv')

In [3]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.00,0,0,0,low,Low Floor
1,flat,sector 89,0.95,2,2,2,New Property,1226.00,1,0,0,low,Mid Floor
2,flat,sector 86,0.46,2,2,1,Under Construction,681.41,0,0,0,low,Low Floor
3,flat,sohna road,0.32,2,2,1,New Property,1000.00,0,0,0,low,High Floor
4,flat,sector 92,1.60,3,4,3+,Relatively New,1615.00,1,0,1,high,Mid Floor


In [4]:
df['furnishing_type'].value_counts()

furnishing_type
0    2359
1    1019
2     188
Name: count, dtype: int64

In [5]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished', 1.0:'semifurnished', 2.0: 'furnished'})

In [6]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.00,0,0,unfurnished,low,Low Floor
1,flat,sector 89,0.95,2,2,2,New Property,1226.00,1,0,unfurnished,low,Mid Floor
2,flat,sector 86,0.46,2,2,1,Under Construction,681.41,0,0,unfurnished,low,Low Floor
3,flat,sohna road,0.32,2,2,1,New Property,1000.00,0,0,unfurnished,low,High Floor
4,flat,sector 92,1.60,3,4,3+,Relatively New,1615.00,1,0,semifurnished,high,Mid Floor


In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
#Applying the Log1p transformation to the target variable
y_transformed = np.log1p(y)

## Ordinal Encoding

In [9]:
columns_to_encode = ['property_type','sector', 'balcony','agePossession','furnishing_type','luxury_category','floor_category']

In [10]:
#creating a column transformr for preprocessing

preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['bedRoom','bathroom','built_up_area','servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder= 'passthrough'
)

In [11]:
#Creating a Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',LinearRegression())
])

In [12]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [13]:
scores.mean(), scores.std()

(0.7379743610501018, 0.03188994221867147)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state=42)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', LinearRegression())])

In [16]:
y_pred = pipeline.predict(X_test)

In [17]:
y_pred = np.expm1(y_pred)

In [18]:
mean_absolute_error(np.expm1(y_test), y_pred)

0.9489148390873442

In [19]:
def scorer(model_name, model):
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocesor', preprocessor),
        ('regressor', model)
    ])
    
    # k-fold cross validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring= 'r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test), y_pred))
    
    return output

In [20]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [21]:
model_output = []
for model_name, model in model_dict.items():
    model_output.append(scorer(model_name, model))

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440

In [22]:
model_output

[['linear_reg', 0.7379743610501018, 0.9489148390873442],
 ['svr', 0.767156007124574, 0.8587483788690616],
 ['ridge', 0.7379800165008699, 0.9488744128788942],
 ['LASSO', 0.05957719557394592, 1.5594937768214137],
 ['decision tree', 0.7908701469844501, 0.6094165697175464],
 ['random forest', 0.8844657085750163, 0.5204842964121501],
 ['extra trees', 0.8734218740621481, 0.5524136883088431],
 ['gradient boosting', 0.8774625745193468, 0.5569954545241171],
 ['adaboost', 0.7520811001737612, 0.8480337664116868],
 ['mlp', 0.8095416006017253, 0.7354781490894652],
 ['xgboost', 0.8972404554202316, 0.48235851867943585]]

In [23]:
model_df = pd.DataFrame(model_output, columns=['name', 'r2','mae'])

In [24]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.897240,0.482359
5,random forest,0.884466,0.520484
6,extra trees,0.873422,0.552414
7,gradient boosting,0.877463,0.556995
4,decision tree,0.790870,0.609417
9,mlp,0.809542,0.735478
8,adaboost,0.752081,0.848034
1,svr,0.767156,0.858748
2,ridge,0.737980,0.948874
0,linear_reg,0.737974,0.948915


## OneHotEncoding

In [25]:
#creating a column transformer for preprocessing
columns_to_encode = ['property_type', 'balcony','agePossession','furnishing_type','luxury_category','floor_category']
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['bedRoom', 'bathroom','built_up_area', 'servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop='first'), ['sector', 'agePossession', 'furnishing_type'])
        ],
        remainder= 'passthrough'
    )

In [26]:
#Creating a Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [27]:
# K fold cross validation

kfold = KFold(n_splits= 10, shuffle=True, random_state= 42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [28]:
scores.mean()

0.8576438954198571

In [29]:
scores.std()

0.01692543275195749

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state= 42)

In [31]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'agePossession',
                                                   'furnishing_type'])])),
                ('regressor', LinearRegression())])

In [32]:
y_pred = pipeline.predict(X_test)

In [33]:
y_pred = np.expm1(y_pred)

In [34]:
mean_absolute_error(np.expm1(y_test), y_pred)

0.6792869102491023

In [35]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocesor', preprocessor),
        ('regressor', model)
        ])
    
    # K fold cross validation
    kfold =  KFold(n_splits= 10, shuffle= True, random_state= 42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state= 42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test), y_pred))
    
    return output

    

In [36]:
model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest': RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': XGBRegressor()
}

In [37]:
model_output = []
for model_name, model in model_dict.items():
    model_output.append(scorer(model_name, model))    

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440

In [38]:
model_df = pd.DataFrame(model_output, columns=['name', 'r2', 'mae'])

In [39]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.889918,0.458476
6,extra trees,0.888932,0.492946
5,random forest,0.876730,0.530016
9,mlp,0.874623,0.538358
1,svr,0.881617,0.550129
7,gradient boosting,0.860882,0.586576
0,linear_reg,0.857644,0.679287
2,ridge,0.857912,0.682025
4,decision tree,0.779529,0.685947
8,adaboost,0.731967,0.901762


## One HotEncoding With PCA

In [40]:
#Creating a column transfer for preprocessing
columns_to_encode = ['property_type', 'balcony','agePossession','furnishing_type','luxury_category','floor_category']
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['bedRoom', 'bathroom','built_up_area', 'servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop= 'first', sparse_output= False), ['sector', 'agePossession'])
    ],
    remainder = 'passthrough'
)

In [41]:
#creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca',PCA(n_components= 0.95)),
    ('regressor', LinearRegression())
])

In [42]:
#k-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')


In [43]:
scores.mean()

0.76454811272919

In [44]:
scores.std()

0.0276877504803559

In [45]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocesor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
        ])
    
    # K fold cross validation
    kfold =  KFold(n_splits= 10, shuffle= True, random_state= 42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state= 42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test), y_pred))
    
    return output

In [46]:
model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest': RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': XGBRegressor()
}

In [47]:
model_output = []
for model_name, model in model_dict.items():
    model_output.append(scorer(model_name, model))    

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Rahul H

In [48]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [49]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.839079,0.622709
10,xgboost,0.824851,0.636093
5,random forest,0.824909,0.647847
9,mlp,0.818275,0.693123
7,gradient boosting,0.818487,0.695007
1,svr,0.819969,0.716029
4,decision tree,0.627348,0.879194
2,ridge,0.764591,0.893958
0,linear_reg,0.764548,0.894108
8,adaboost,0.699157,0.926368


## Target Encoder

In [50]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [51]:
import category_encoders as ce

columns_to_encode = ['property_type', 'balcony','agePossession','furnishing_type','luxury_category',
                     'floor_category']

# Creating a column Transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area','servant room','store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop='first', sparse_output=False), ['agePossession']),
        ('target_enc', ce.TargetEncoder(),['sector'])        
    ],
    remainder= 'passthrough')


In [52]:
#Creating Pipeline

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [53]:
# K fold cross validation
kfold = KFold(n_splits= 10, shuffle= True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv =kfold, scoring='r2')

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

In [54]:
scores.mean(), scores.std()

(0.8307821548460954, 0.019252637339234387)

In [55]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocesor', preprocessor),
        ('regressor', model)
        ])
    
    # K fold cross validation
    kfold =  KFold(n_splits= 10, shuffle= True, random_state= 42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state= 42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test), y_pred))
    
    return output

In [56]:
model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'LASSO': Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest': RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': XGBRegressor()
}

In [57]:
model_output = []
for model_name, model in model_dict.items():
    model_output.append(scorer(model_name, model))

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

...

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

In [58]:
model_df = pd.DataFrame(model_output, columns=['name', 'r2', 'mae'])

In [60]:
model_df.sort_values(['mae'])

,name,r2,mae
10,xgboost,0.900117,0.468633
5,random forest,0.896937,0.480300
6,extra trees,0.894323,0.509917
7,gradient boosting,0.886211,0.538593
4,decision tree,0.810184,0.613495
9,mlp,0.852962,0.619830
1,svr,0.865843,0.625607
8,adaboost,0.813897,0.747249
0,linear_reg,0.830782,0.755433
2,ridge,0.830797,0.755750


## HyperParameter Tuning

In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}



columns_to_encode = ['property_type', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)






pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])





kfold = KFold(n_splits=10, shuffle=True, random_state=42)
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)
search.fit(X, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
640 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
237 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['property_type',
                                                                          'balcony',
                                                                          'agePossession',
                                                                          'furnishing_type',
                                                                          'luxury_c...
                                                                         OneHotEncoder(drop='first',
                                                                                       sparse_output=False),
                                                                         ['agePossession']),
                                                                        ('target_enc',
                                                                         TargetEncoder(),
                                                                         ['sector'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__max_features': ['auto', 'sqrt'],
                         'regressor__max_samples': [0.1, 0.25, 0.5, 1.0],
                         'regressor__n_estimators': [50, 100, 200, 300]},
             scoring='r2', verbose=4)

In [63]:
final_pipe = search.best_estimator_

In [64]:
search.best_params_

{'regressor__max_depth': 30,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [65]:
search.best_score_

0.8949604144006125

In [66]:
final_pipe.fit(X,y_transformed)

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future v

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['agePossession']),
                                                 ('target_enc', TargetEncoder(),
                                                  ['sector'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=30, max_features='sqrt',
                                       max_samples=1.0, n_estimators=300))])

In [70]:
# Define your preprocessing steps
columns_to_encode = ['property_type', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop='first', sparse=False), ['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ],
    remainder='passthrough'
)

# Create the XGBoost regressor
xgb_model = XGBRegressor()

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

# Define the hyperparameters to tune
param_grid = {
    'regressor__n_estimators': [100, 200, 300, 400, 500, 600],
    'regressor__max_depth': [3, 4, 5],
    'regressor__learning_rate': [0.05, 0.1, 0.2],
    'regressor__subsample': [0.8, 0.9, 1.0],
}

# Create K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform GridSearchCV with the pipeline
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)
search.fit(X, y_transformed)

Fitting 10 folds for each of 162 candidates, totalling 1620 fits


C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in 

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['bedRoom',
                                                                          'bathroom',
                                                                          'built_up_area',
                                                                          'servant '
                                                                          'room',
                                                                          'store '
                                                                          'room']),
                                                                        ('cat',
                                                                         OrdinalEncoder(),
                                                                         ['property_type',
                                                                          'sector',
                                                                          'balcony',
                                                                          'agePossession',
                                                                          'furnishing_type',...
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     n_estimators=100,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     predictor=None,
                                                     random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'regressor__learning_rate': [0.05, 0.1, 0.2],
                         'regressor__max_depth': [3, 4, 5],
                         'regressor__n_estimators': [100, 200, 300, 400, 500,
                                                     600],
                         'regressor__subsample': [0.8, 0.9, 1.0]},
             scoring='r2', verbose=4)

In [71]:
search.best_params_

{'regressor__learning_rate': 0.05,
 'regressor__max_depth': 5,
 'regressor__n_estimators': 600,
 'regressor__subsample': 0.9}

In [72]:
search.best_score_

0.9096222339371138

In [73]:
final1_pipe = search.best_estimator_

In [74]:
final1_pipe.fit(X,y_transformed)

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=600,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [76]:
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# # Split your data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# # Define your preprocessing steps
# columns_to_encode = ['property_type', 'sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
#         ('cat', OrdinalEncoder(), columns_to_encode),
#         ('cat1', OneHotEncoder(drop='first', sparse=False), ['agePossession']),
#         ('target_enc', ce.TargetEncoder(), ['sector'])
#     ],
#     remainder='passthrough'
# )
# def objective(params):
#     xgb_model = xgb.XGBRegressor(
#         n_estimators=int(params['n_estimators']),
#         max_depth=int(params['max_depth']),
#         learning_rate=params['learning_rate'],
#         subsample=params['subsample'],
#         random_state=42,
#     )
#     pipeline = Pipeline([
#         ('preprocessor', preprocessor),
#         ('regressor', xgb_model)
#     ])
#     score = cross_val_score(pipeline, X_train, y_train, scoring='r2', cv=5).mean()
#     return {'loss': -score, 'status': STATUS_OK}  # Note the negative sign for 'loss'

# # Define the search space for hyperparameters
# space = {
#     'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
#     'max_depth': hp.quniform('max_depth', 3, 15, 1),
#     'learning_rate': hp.loguniform('learning_rate', -5, 0),
#     'subsample': hp.uniform('subsample', 0.5, 1),
# }

# # Perform Bayesian optimization
# trials = Trials()
# best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# # Retrieve the best hyperparameters
# best_hyperparameters = {
#     'n_estimators': int(best['n_estimators']),
#     'max_depth': int(best['max_depth']),
#     'learning_rate': best['learning_rate'],
#     'subsample': best['subsample'],
# }

# # Train the final XGBoost model with the best hyperparameters
# final_xgb_model = xgb.XGBRegressor(**best_hyperparameters, random_state=42)
# final_xgb_model.fit(X_train, y_train)

### Exporting the Model

In [91]:
columns_to_encode = ['property_type', 'balcony','agePossession','furnishing_type','luxury_category','floor_category']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1', OneHotEncoder(drop='first', sparse=False), ['sector','agePossession']),
        
    ],
    remainder='passthrough'
)

In [92]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=500))
])

In [93]:
pipeline.fit(X,y_transformed)

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('cat1',
                                                  OneHotEncoder(drop='first',
                                                                sparse...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=500,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size= 0.2, random_state= 42)
pipeline.fit(X_train, y_train)
    
y_pred = pipeline.predict(X_test)
    
y_pred = np.expm1(y_pred)
    
mean_absolute_error(np.expm1(y_test), y_pred)

C:\Users\Rahul Harshawardhan\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.47683070515133746

In [94]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [95]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [96]:
X

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,3,2,2,New Property,850.00,0,0,unfurnished,low,Low Floor
1,flat,sector 89,2,2,2,New Property,1226.00,1,0,unfurnished,low,Mid Floor
2,flat,sector 86,2,2,1,Under Construction,681.41,0,0,unfurnished,low,Low Floor
3,flat,sohna road,2,2,1,New Property,1000.00,0,0,unfurnished,low,High Floor
4,flat,sector 92,3,4,3+,Relatively New,1615.00,1,0,semifurnished,high,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3561,flat,sector 84,2,2,1,Relatively New,532.00,0,0,unfurnished,medium,Mid Floor
3562,house,sector 109,5,5,3+,Relatively New,6228.00,1,1,unfurnished,high,Low Floor
3563,flat,sector 2,1,1,1,Moderately Old,665.00,0,0,semifurnished,medium,Mid Floor
3564,house,sector 43,5,6,3,Moderately Old,5490.00,1,1,unfurnished,medium,Mid Floor


## Trying Out Predictions

In [97]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [98]:
X.iloc[0].values

array(['flat', 'sector 36', 3, 2, '2', 'New Property', 850.0, 0, 0,
       'unfurnished', 'low', 'Low Floor'], dtype=object)

In [103]:
data = [['house',  'sector 102', 4, 3, '3', 'New Property', 2750, 0, 0, 'unfurnished', 'low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 49,4,3,3,New Property,2750,0,0,unfurnished,low,Low Floor


In [104]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3566 entries, 0 to 3565
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3566 non-null   object 
 1   sector           3566 non-null   object 
 2   bedRoom          3566 non-null   int64  
 3   bathroom         3566 non-null   int64  
 4   balcony          3566 non-null   object 
 5   agePossession    3566 non-null   object 
 6   built_up_area    3566 non-null   float64
 7   servant room     3566 non-null   int64  
 8   store room       3566 non-null   int64  
 9   furnishing_type  3566 non-null   object 
 10  luxury_category  3566 non-null   object 
 11  floor_category   3566 non-null   object 
dtypes: float64(1), int64(4), object(7)
memory usage: 334.4+ KB


In [106]:
np.expm1(pipeline.predict(one_df))

array([3.958084], dtype=float32)

In [ ]:
X.dtypes

In [96]:
y_transformed.isnull().sum()

0

In [91]:
sorted(X['sector'].unique().tolist())

['dwarka expressway',
 'gwal pahari',
 'manesar',
 'sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 37d',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 63a',
 'sector 65',
 'sector 66',
 'sector 67',
 'se